# Using ezancestry as a Python library

In [1]:
from pathlib import Path

from sklearn.model_selection import train_test_split

In [2]:
# load config variables
from ezancestry.config import aisnps_directory as _aisnps_directory
from ezancestry.config import aisnps_set as _aisnps_set
from ezancestry.config import algorithm as _algorithm
from ezancestry.config import k as _k
from ezancestry.config import models_directory as _models_directory
from ezancestry.config import n_components as _n_components
from ezancestry.config import population_level as _population_level
from ezancestry.config import samples_directory as _samples_directory
from ezancestry.config import thousand_genomes_directory as _thousand_genomes_directory

# load functions
from ezancestry.aisnps import extract_aisnps
from ezancestry.dimred import dimensionality_reduction
from ezancestry.evaluate import export_performance
from ezancestry.fetch import download_thousand_genomes
from ezancestry.model import predict_ancestry, train
from ezancestry.process import (encode_genotypes, get_1kg_labels,
                                process_user_input, vcf2df)

### pull aisnps from 1kG

In [3]:
# # kidd
aisnps_file = Path(_aisnps_directory).joinpath("kidd.aisnp.txt")
#extract_aisnps(_thousand_genomes_directory, aisnps_file, aisnps_set="kidd")


In [4]:
# # Seldin#
aisnps_file = Path(_aisnps_directory).joinpath("Seldin.aisnp.txt")
#extract_aisnps(_thousand_genomes_directory, aisnps_file, aisnps_set="Seldin")

In [5]:
# pull the 1000 Genomes Project samples
dfsamples = get_1kg_labels(_samples_directory)

In [6]:
dfsamples.head()

,population,superpopulation,gender
sample,,,
HG00096,GBR,EUR,male
HG00097,GBR,EUR,female
HG00099,GBR,EUR,female
HG00100,GBR,EUR,female
HG00101,GBR,EUR,male


In [7]:
vcf_fname = Path(_aisnps_directory).joinpath("kidd.aisnp.1kG.vcf")
df_kidd = vcf2df(vcf_fname, dfsamples)

df_kidd.head()

,rs3737576,rs7554936,rs2814778,rs4918664,rs174570,rs1079597,rs2238151,rs671,rs7997709,rs1572018,...,rs3823159,rs917115,rs1462906,rs6990312,rs2196051,rs1871534,rs3814134,population,superpopulation,gender
HG00096,TT,CT,TT,AG,CC,CC,TT,GG,TT,CC,...,AA,TT,CC,GG,AA,CC,AA,GBR,EUR,male
HG00097,TT,TT,TT,AA,CC,CC,CC,GG,CT,CC,...,AA,TT,CC,GT,AG,CC,AA,GBR,EUR,female
HG00099,TT,CT,TT,AA,CC,CC,CT,GG,TT,CC,...,AA,TT,CC,GG,AG,CC,AA,GBR,EUR,female
HG00100,TT,TT,TT,AG,CT,CC,TT,GG,TT,CC,...,AA,TT,CC,GT,AG,CC,AA,GBR,EUR,female
HG00101,TT,CT,TT,AA,CC,CC,TT,GG,TT,CC,...,AA,CT,CC,GG,AA,CC,AA,GBR,EUR,male


In [8]:
vcf_fname = Path(_aisnps_directory).joinpath("Seldin.aisnp.1kG.vcf")
df_seldin = vcf2df(vcf_fname, dfsamples)

/Users/jacksonc08/ezancestry/ezancestry/process.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[variant.ID] = [
/Users/jacksonc08/ezancestry/ezancestry/process.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[variant.ID] = [
/Users/jacksonc08/ezancestry/ezancestry/process.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented

### could start here

In [9]:
train_kidd, test_kidd, y_train_kidd, y_test_kidd = train_test_split(
    df_kidd,
    df_kidd["superpopulation"],
    test_size=0.2,
    stratify=df_kidd["superpopulation"],
    random_state=42,
)

### one-hot encode snps

In [10]:
# The user could have missing snps
df_user = df_kidd[df_kidd.columns[0:43]].copy()

# The user could have extra snps
df_user["extra_snp"] = "TT"

# The user could have genotypes that weren't in the original encoder
df_user.loc["HG00096", "rs3737576"] = "blah"

In [11]:
df_user.head()

,rs3737576,rs7554936,rs2814778,rs4918664,rs174570,rs1079597,rs2238151,rs671,rs7997709,rs1572018,...,rs260690,rs6754311,rs10497191,rs310644,rs2024566,rs12498138,rs4833103,rs1229984,rs3811801,extra_snp
HG00096,blah,CT,TT,AG,CC,CC,TT,GG,TT,CC,...,AC,TT,CC,CT,AA,GG,AC,CC,GG,TT
HG00097,TT,TT,TT,AA,CC,CC,CC,GG,CT,CC,...,CC,TT,CC,TT,AG,GG,AC,CC,GG,TT
HG00099,TT,CT,TT,AA,CC,CC,CT,GG,TT,CC,...,AA,TT,CC,TT,AG,GG,AC,CC,GG,TT
HG00100,TT,TT,TT,AG,CT,CC,TT,GG,TT,CC,...,AA,CT,CC,TT,AA,AG,AA,CC,GG,TT
HG00101,TT,CT,TT,AA,CC,CC,TT,GG,TT,CC,...,AA,CC,CC,TT,AG,AG,AC,CC,GG,TT


In [12]:
ohe_user = encode_genotypes(df_user, aisnps_set="kidd", overwrite_encoder=False)

2022-10-25 13:56:20.037 | INFO     | ezancestry.process:encode_genotypes:137 - Successfully loaded an encoder from /Users/jacksonc08/.ezancestry/data/models/one_hot_encoder.kidd.bin


In [13]:
ohe_user

,rs3737576_CC,rs3737576_CT,rs3737576_TT,rs7554936_CC,rs7554936_CT,rs7554936_TT,rs2814778_CC,rs2814778_CT,rs2814778_TT,rs798443_AA,...,rs4891825_GG,rs7251928_AA,rs7251928_AC,rs7251928_CC,rs310644_CC,rs310644_CT,rs310644_TT,rs2024566_AA,rs2024566_AG,rs2024566_GG
HG00096,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
HG00097,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
HG00099,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
HG00100,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
HG00101,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
NA21141,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
NA21142,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
NA21143,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [14]:
# make sure "blah" genotype didn't get encoded
ohe_user.loc["HG00096", ["rs3737576_CC", "rs3737576_CT", "rs3737576_TT"]]

rs3737576_CC    0.0
rs3737576_CT    0.0
rs3737576_TT    0.0
Name: HG00096, dtype: float64

In [15]:
# change to True to write new encoders
OVERWRITE_ENCODER = False

In [16]:
# get an encoder for each snp set
df_kidd_encoded = encode_genotypes(df_kidd, aisnps_set="kidd", overwrite_encoder=OVERWRITE_ENCODER)
df_seldin_encoded = encode_genotypes(df_seldin, aisnps_set="Seldin", overwrite_encoder=OVERWRITE_ENCODER)

2022-10-25 13:56:20.120 | INFO     | ezancestry.process:encode_genotypes:137 - Successfully loaded an encoder from /Users/jacksonc08/.ezancestry/data/models/one_hot_encoder.kidd.bin
2022-10-25 13:56:20.171 | INFO     | ezancestry.process:encode_genotypes:137 - Successfully loaded an encoder from /Users/jacksonc08/.ezancestry/data/models/one_hot_encoder.seldin.bin


In [17]:
df_kidd_encoded.head()

,rs3737576_CC,rs3737576_CT,rs3737576_TT,rs7554936_CC,rs7554936_CT,rs7554936_TT,rs2814778_CC,rs2814778_CT,rs2814778_TT,rs798443_AA,...,rs4891825_GG,rs7251928_AA,rs7251928_AC,rs7251928_CC,rs310644_CC,rs310644_CT,rs310644_TT,rs2024566_AA,rs2024566_AG,rs2024566_GG
HG00096,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
HG00097,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
HG00099,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
HG00100,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
HG00101,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


### dimensionality reduction & training

In [18]:
OVERWRITE_MODEL = False

In [19]:
# write all the super population dimred models for kidd and Seldin
for aisnps_set, df, df_labels in zip(
    ["kidd", "Seldin"], 
    [df_kidd_encoded, df_seldin_encoded], 
    [df_kidd["superpopulation"], df_seldin["superpopulation"]]
):
    for algorithm, labels in zip(["pca", "umap", "nca"], [None, None, None, df_labels]):
        print(algorithm,aisnps_set,OVERWRITE_MODEL,labels)
        df_reduced = dimensionality_reduction(df, algorithm="nca", aisnps_set=aisnps_set, overwrite_model=OVERWRITE_MODEL, labels=labels, population_level="super population")
        knn_model = train(df_reduced, df_labels, algorithm="nca", aisnps_set=aisnps_set, k=9, population_level="superpopulation", overwrite_model=OVERWRITE_MODEL)

2022-10-25 13:56:20.254 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successfully loaded a dimensionality reduction model
2022-10-25 13:56:20.260 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successfully loaded a dimensionality reduction model
2022-10-25 13:56:20.266 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successfully loaded a dimensionality reduction model
2022-10-25 13:56:20.275 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successfully loaded a dimensionality reduction model
2022-10-25 13:56:20.283 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successfully loaded a dimensionality reduction model
2022-10-25 13:56:20.290 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successfully loaded a dimensionality reduction model


pca kidd False None
umap kidd False None
nca kidd False None
pca Seldin False None
umap Seldin False None
nca Seldin False None


In [20]:
# write all the population dimred models for kidd and Seldin
for aisnps_set, df, df_labels in zip(
    ["kidd", "Seldin"], 
    [df_kidd_encoded, df_seldin_encoded], 
    [df_kidd["population"], df_seldin["population"]]
):
    for algorithm, labels in zip(["nca"], [df_labels]):
        df_reduced = dimensionality_reduction(df, algorithm=algorithm, aisnps_set=aisnps_set, overwrite_model=OVERWRITE_MODEL, labels=labels, population_level="population")
        knn_model = train(df_reduced, labels, algorithm=algorithm, aisnps_set=aisnps_set, k=9, population_level="population", overwrite_model=OVERWRITE_MODEL)

2022-10-25 13:56:20.302 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successfully loaded a dimensionality reduction model
2022-10-25 13:56:20.311 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successfully loaded a dimensionality reduction model


# Predict

In [21]:
from ezancestry.commands import predict

In [22]:
from snps import SNPs

In [23]:
mygenomefile = "/Users/jacksonc08/ezancestry/data/folder/remapped_HaplotypeCaller_subset.kidd.vcf.gz"

## load from DataFrame

In [24]:
# the snps Python package will read the genome file properly 
mygenome = SNPs(mygenomefile)
mygenomedf = mygenome.snps

In [25]:
len(mygenomedf)

28

In [26]:
mygenomedf.head()

,chrom,pos,genotype
rsid,,,
rs7554936,1,151122489,TT
rs798443,2,7968275,GA
rs1876482,2,17362568,GA
rs260690,2,109579738,AA
rs6754311,2,136707982,TC


In [27]:
# predict on the 
predictions = predict(mygenomedf, 
                    aisnps_set="kidd",
                    k=None,
                    n_components=None,
                    algorithm=None,
                    write_predictions=False,
                    models_directory=None,
                    output_directory=None,
                    aisnps_directory=None,
                    thousand_genomes_directory=None,
                    samples_directory=None
                     )

2022-10-25 13:56:21.293 | INFO     | ezancestry.process:_input_to_dataframe:285 - sample has a valid genotype for 0 out of a possible 55 (0.0%)
2022-10-25 13:56:21.304 | INFO     | ezancestry.process:encode_genotypes:137 - Successfully loaded an encoder from /Users/jacksonc08/.ezancestry/data/models/one_hot_encoder.kidd.bin
2022-10-25 13:56:21.307 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successfully loaded a dimensionality reduction model
2022-10-25 13:56:21.309 | INFO     | ezancestry.model:predict_ancestry:94 - Successfully loaded trained knn model: /Users/jacksonc08/.ezancestry/data/models/knn.pca.kidd.population.bin
/Users/jacksonc08/ezancestry/ezancestry/commands.py:312: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pop_predictions = pop_predictions.append(pop_level_predictions)
2022-10-25 13:56:21.383 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successf

In [28]:
predictions

,component1,component2,component3,predicted_population_population,ACB,ASW,BEB,CDX,CEU,CHB,...,TSI,YRI,predicted_population_superpopulation,AFR,AMR,EAS,EUR,SAS,population_description,superpopulation_name
sample,0.118744,0.153,0.326515,ITU,0.0,0.0,0.089197,0.0,0.0,0.0,...,0.0,0.0,SAS,0.0,0.172022,0.0,0.0,0.827978,Indian Telugu in the UK,South Asian Ancestry


## or load directly from a file

In [29]:
predictions = predict(mygenomefile, 
                    aisnps_set="kidd",
                    k=None,
                    n_components=None,
                    algorithm="nca",
                    write_predictions=False,
                    models_directory=None,
                    output_directory=None,
                    aisnps_directory=None,
                    thousand_genomes_directory=None,
                    samples_directory=None
                     )

2022-10-25 13:56:21.436 | INFO     | ezancestry.process:_input_to_dataframe:285 - remapped_HaplotypeCaller_subset.kidd.vcf.gz has a valid genotype for 0 out of a possible 55 (0.0%)
2022-10-25 13:56:21.445 | INFO     | ezancestry.process:encode_genotypes:137 - Successfully loaded an encoder from /Users/jacksonc08/.ezancestry/data/models/one_hot_encoder.kidd.bin
2022-10-25 13:56:21.449 | INFO     | ezancestry.dimred:dimensionality_reduction:126 - Successfully loaded a dimensionality reduction model
2022-10-25 13:56:21.451 | INFO     | ezancestry.model:predict_ancestry:94 - Successfully loaded trained knn model: /Users/jacksonc08/.ezancestry/data/models/knn.nca.kidd.population.bin
/Users/jacksonc08/ezancestry/ezancestry/commands.py:312: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pop_predictions = pop_predictions.append(pop_level_predictions)
2022-10-25 13:56:21.465 | INFO     | ezancestry.dimred:di

In [30]:
predictions

,component1,component2,component3,predicted_population_population,ACB,ASW,BEB,CDX,CEU,CHB,...,TSI,YRI,predicted_population_superpopulation,AFR,AMR,EAS,EUR,SAS,population_description,superpopulation_name
remapped_HaplotypeCaller_subset.kidd.vcf.gz,0.0,0.0,0.0,MXL,0.0,0.120935,0.0,0.0,0.0,0.0,...,0.0,0.0,AMR,0.0,1.0,0.0,0.0,0.0,"Mexican Ancestry in Los Angeles, California",American Ancestry


In [31]:
from ezancestry.commands import plot


In [32]:
# the snps Python package will read the genome file properly 
predictionsfile = "/Users/jacksonc08/ezancestry/ezancestry/predictions.csv"
predictfile = predfile.snps

NameError: name 'predfile' is not defined

In [ ]:
plotm = plot("/Users/jacksonc08/ezancestry/ezancestry/predictions.csv")

In [ ]:
plotm